# Running deeptfactor on MAGs and Genomes from other studies

In [1]:
import os

workDir = '/home/sam/FullCyc_metagenome/annotation/deepTfactor'
protein_fasta = '/home/sam/FullCyc_metagenome/annotation/IMG/Ga0334612_proteins.faa'
nproc = 20

In [2]:
import pandas as pd
from multiprocessing import Pool
import time
from Bio import SeqIO
import re



## Running deeptfactor

In [3]:
outprefix = os.path.join(workDir, 'deepTfactor_output', re.sub('.faa$', '.TFs', os.path.split(protein_fasta)[1]))
outprefix

'/home/sam/FullCyc_metagenome/annotation/deepTfactor/deepTfactor_output/Ga0334612_proteins.TFs'

In [ ]:
start_time = time.time()
print('Process started at ' + str(start_time))
cmd = ' '.join(['python /home/sam/new_repo/deeptfactor/tf_running.py',
                '-i', protein_fasta,
                '-o', outprefix,
                '-ckpt /home/sam/new_repo/deeptfactor/trained_model/DeepTFactor_ckpt.pt',
                '-cpu', str(nproc)])
!$cmd
proc_time = time.time() - start_time

Process started at 1624031625.5958176


In [7]:
print(str(proc_time))

110824.1569647789


In [8]:
print('Done!')

Done!


## Get genome info

In [36]:
study = 'rainfall'
genome = 'GCA_005881935.1_ASM588193v1_protein.faa'
deeptfactor_df = pd.read_csv(os.path.join(workDir, study, 'deeptfactor_results', genome.replace('.faa', '_TF_results'), 'prediction_result.txt'),
                             sep = '\t', dtype={'sequence_ID': str, 'prediction': str, 'score': float})
TF_count = len(list(set(deeptfactor_df[deeptfactor_df['prediction'] == 'True']['sequence_ID'])))
TF_count


91

In [41]:
MAG_sum_df = pd.DataFrame(columns=['Study', 'Genome', 'Accession', 'Gene_count', 'Transcription_factors', 
                                   'Total_Secreted_genes', 'Gram_Pos_Secreted_genes', 'Gram_Neg_Secreted_genes',
                                   'Taxonomy', 'Organism'])

study_list = ['amazon_deforestation', 'biocharSIP', 'rainfall', 'DeepSIP', 'SIPLigCel', 'Avena_rhizosphere', 'SIPrhizosphere']


for study in study_list:
    genome_list = [f for f in os.listdir(os.path.join(workDir, study, 'protein_faa')) if f.endswith('.faa')]

    for genome in genome_list:
        gene_count = 0
        trans_fact_count = 0
        for record in SeqIO.parse(os.path.join(workDir, study, 'protein_faa', genome), 'fasta'):
            gene_count += 1
        
        deeptfactor_df = pd.read_csv(os.path.join(workDir, study, 'deeptfactor_results', genome.replace('.faa', '_TF_results'), 'prediction_result.txt'),
                                     sep = '\t', dtype={'sequence_ID': str, 'prediction': str, 'score': float})
        trans_fact_count = len(list(set(deeptfactor_df[deeptfactor_df['prediction'] == 'True']['sequence_ID'])))

        signalp_grampos = pd.read_csv(os.path.join(workDir, study, 'signalp_output', genome.replace('.faa', '_signalp_gram_pos_summary.signalp5')),
                                      sep = '\t', skiprows = 1)
        signalp_gramneg = pd.read_csv(os.path.join(workDir, study, 'signalp_output', genome.replace('.faa', '_signalp_gram_neg_summary.signalp5')),
                                      sep = '\t', skiprows = 1)
        total_secreted_count = len(list(set(list(signalp_grampos[signalp_grampos['Prediction'] != 'OTHER']['# ID']) + list(signalp_gramneg[signalp_gramneg['Prediction'] != 'OTHER']['# ID']))))
        grampos_secreted_count = len(list(signalp_grampos[signalp_grampos['Prediction'] != 'OTHER']['# ID']))
        gramneg_secreted_count = len(list(signalp_gramneg[signalp_gramneg['Prediction'] != 'OTHER']['# ID']))

        genome_dict = {'Study': study,
                       'Genome': 'NA',
                       'Accession': genome.replace('.faa', ''),
                       'Gene_count': gene_count,
                       'Transcription_factors': trans_fact_count,
                       'Total_Secreted_genes': total_secreted_count,
                       'Gram_Pos_Secreted_genes': grampos_secreted_count,
                       'Gram_Neg_Secreted_genes': gramneg_secreted_count,
                       'Taxonomy': 'NA',
                       'Organism': 'NA'}

        MAG_sum_df = MAG_sum_df.append(genome_dict, ignore_index=True)
    
    

In [42]:
MAG_sum_df

,Study,Genome,Accession,Gene_count,Transcription_factors,Total_Secreted_genes,Gram_Pos_Secreted_genes,Gram_Neg_Secreted_genes,Taxonomy,Organism
0,amazon_deforestation,NA,GCA_003175695.1_ASM317569v1_protein,4168,126,980,928,933,NA,NA
1,amazon_deforestation,NA,GCA_003175705.1_ASM317570v1_protein,3224,114,470,433,422,NA,NA
2,amazon_deforestation,NA,GCA_003175735.1_ASM317573v1_protein,1987,84,341,328,313,NA,NA
3,amazon_deforestation,NA,GCA_003176015.1_ASM317601v1_protein,1006,29,115,106,100,NA,NA
4,amazon_deforestation,NA,GCA_003175865.1_ASM317586v1_protein,3013,110,727,706,645,NA,NA
...,...,...,...,...,...,...,...,...,...,...
1410,SIPrhizosphere,NA,MAG41,4274,233,941,894,881,NA,NA
1411,SIPrhizosphere,NA,MAG40,2465,74,475,452,442,NA,NA
1412,SIPrhizosphere,NA,MAG33,4039,151,1039,997,981,NA,NA
1413,SIPrhizosphere,NA,MAG52,1335,41,220,206,207,NA,NA


In [43]:
MAG_sum_df.to_csv(os.path.join(workDir, 'transfact_secrete_table_new.txt'), sep='\t', index=False)

## List genes that were not run due to ambiguous bases

In [3]:
study_list = ['amazon_deforestation', 'biocharSIP', 'rainfall', 'DeepSIP', 
              'SIPLigCel', 'Avena_rhizosphere', 'SIPrhizosphere']
genome_list = []
for study in study_list:
    genome_list = genome_list + [os.path.join(workDir, study, 'protein_faa', f) for f in os.listdir(os.path.join(workDir, study, 'protein_faa')) if f.endswith('.faa')]
genome_list


['/home/sam/FullCyc_metagenome/Other_studies_comp/amazon_deforestation/protein_faa/GCA_003175695.1_ASM317569v1_protein.faa',
 '/home/sam/FullCyc_metagenome/Other_studies_comp/amazon_deforestation/protein_faa/GCA_003175705.1_ASM317570v1_protein.faa',
 '/home/sam/FullCyc_metagenome/Other_studies_comp/amazon_deforestation/protein_faa/GCA_003175735.1_ASM317573v1_protein.faa',
 '/home/sam/FullCyc_metagenome/Other_studies_comp/amazon_deforestation/protein_faa/GCA_003176015.1_ASM317601v1_protein.faa',
 '/home/sam/FullCyc_metagenome/Other_studies_comp/amazon_deforestation/protein_faa/GCA_003175865.1_ASM317586v1_protein.faa',
 '/home/sam/FullCyc_metagenome/Other_studies_comp/amazon_deforestation/protein_faa/GCA_003175815.1_ASM317581v1_protein.faa',
 '/home/sam/FullCyc_metagenome/Other_studies_comp/amazon_deforestation/protein_faa/GCA_003175925.1_ASM317592v1_protein.faa',
 '/home/sam/FullCyc_metagenome/Other_studies_comp/amazon_deforestation/protein_faa/GCA_003176075.1_ASM317607v1_protein.faa',


In [4]:
ambig_genome_list = []
ambig_gene_id_list = []
ambig_descript_list = []
for genome in genome_list:
    for gene in SeqIO.parse(genome, 'fasta'):
        ambig_count = 0
        ambig_count = len([a for a in ['*', 'U', 'B', 'J', '-'] if a in gene.seq])
        if ambig_count > 0:
            ambig_genome_list.append(genome)
            ambig_gene_id_list.append(gene.id)
            ambig_descript_list.append(gene.description)


In [5]:
ambig_df = pd.DataFrame({'Genome': ambig_genome_list, 'Gene_ID': ambig_gene_id_list, 'Gene_description': ambig_descript_list})
ambig_df.to_csv(os.path.join(workDir, 'ambiguousAA_genes.txt'), sep='\t', index=False)
ambig_df


,Genome,Gene_ID,Gene_description
0,/home/sam/FullCyc_metagenome/Other_studies_com...,PWT84656.1,"PWT84656.1 selenide, water dikinase SelD [Acid..."
1,/home/sam/FullCyc_metagenome/Other_studies_com...,PWT98257.1,PWT98257.1 formate dehydrogenase [Acidobacteri...
2,/home/sam/FullCyc_metagenome/Other_studies_com...,PWU05010.1,"PWU05010.1 selenide, water dikinase SelD [Acid..."
3,/home/sam/FullCyc_metagenome/Other_studies_com...,PWT79105.1,"PWT79105.1 selenide, water dikinase SelD, part..."
4,/home/sam/FullCyc_metagenome/Other_studies_com...,PWU22906.1,PWU22906.1 glycine reductase [Candidatus Rokub...
...,...,...,...
1061,/home/sam/FullCyc_metagenome/Other_studies_com...,TMQ22922.1,TMQ22922.1 alkylhydroperoxidase AhpD family co...
1062,/home/sam/FullCyc_metagenome/Other_studies_com...,TLZ47716.1,TLZ47716.1 formate dehydrogenase-N subunit alp...
1063,/home/sam/FullCyc_metagenome/Other_studies_com...,2558857210,2558857210 N520DRAFT_1408 formate dehydrogenas...
1064,/home/sam/FullCyc_metagenome/Other_studies_com...,2559029334,2559029334 N540DRAFT_1108 formate dehydrogenas...
